### MIKEL GALLO

In [5]:
## Packages
import numpy as np
from numpy.random import binomial, normal, seed

In [5]:
#EXAMPLE - RCT or AB Test

seed (1234)
n = 10000

T = np.random.binomial(1,0.5,size=n)
O = np.random.binomial(1,0.3,size=n)*(T) + np.random.binomial(1,0.5,size=n)*(1-T)
print('Treatment: ', T)
print('Output: ', O)
#condition_prob_diff = mean(outcome[treatment==1]) - mean(outcome[treatment==0])
conditional_prob_diff = np.mean(O[T==1]) - np.mean(O[T==0])
print('Conditional probability diff: ',conditional_prob_diff)
#-0.1810218929047634

#Adjustment
#?

#Theoretical ATE = (E[O|T==1])

eO_t1 = (0.3)*(1) + (0.5)*(1-1)
eO_t0 = (0.3)*(0) + (0.5)*(1-0)

Theoretical_ATE = eO_t1 - eO_t0

print('theoretical ATE: ',Theoretical_ATE)

Treatment:  [0 1 0 ... 0 1 1]
Output:  [1 0 0 ... 0 1 1]
Conditional probability diff:  -0.1810218929047634
theoretical ATE:  -0.2


| #   | Question                                        | Result |
|-----|-------------------------------------------------|--------|
| 1   | Estimated E(O\|T=1) - E(O\|T=0)                 | -0.18  |
| 2   | adjustment(1) - adjustment(0)                   |   ?    |
| 3   | Theoretical ATE = E(O\|do(T=1)) - E(O\|do(T=0)) | -0.2   |
| 4   | Estimated ATE from data                         | -0.18  |

- No need for adjustment
- When doing an AB Test we automatically remove all potential confounders
- It is possible to directly calculate the ATE

### Part 1

In [7]:
#EXERCISE A

seed (1234)
n = 10000

# 𝜀 ~N(0,1)
epsilon = normal(0,1)

C = np.random.binomial(1,0.8,size=n)
#Z := B(0.6) C + B(0.1) (1-C)
Z = np.random.binomial(1,0.6,size=n)*C + np.random.binomial(1,0.1,size=n)*(1-C)
#T := B(0.6) Z + B(0.2)(1-Z)
T = np.random.binomial(1,0.6,size=n)*Z + np.random.binomial(1,0.2,size=n)*(1-Z)
#O := B(0.3) T + B(0.5) (1-T) + C + 𝜀
O = np.random.binomial(1,0.3,size=n)*(T) + np.random.binomial(1,0.5,size=n)*(1-T) + C + epsilon

print('Confounder: ', C)
print('Variable to adjust for: ', Z)
print('Treatment: ', T)
print('Output: ', O)

#condition_prob_diff = mean(outcome[treatment==1]) - mean(outcome[treatment==0])
conditional_prob_diff = np.mean(O[T==1]) - np.mean(O[T==0])
print('Conditional probability diff: ',conditional_prob_diff)

#ADJUSTED FORMULA 

e_t1_z1 = np.mean(O[(T==1)&(Z==1)])

e_t1_z0 = np.mean(O[(T==1)&(Z==0)])

e_t0_z1 = np.mean(O[(T==0)&(Z==1)])

e_t0_z0 = np.mean(O[(T==0)&(Z==0)])

# Calculate the ate
adjusted_formula = (e_t1_z1 * np.mean(Z == 1) + e_t1_z0 * np.mean(Z == 0) ) - ( e_t0_z1 * np.mean(Z == 1) + e_t0_z0 * np.mean(Z == 0))
print('Adjusted Formula: ',adjusted_formula)

#Theoretical ATE 

eO_t1 = (0.3)*(1) + (0.5)*(1-1)
eO_t0 = (0.3)*(0) + (0.5)*(1-0)

Theoretical_ATE = eO_t1 - eO_t0

print('theoretical ATE: ',Theoretical_ATE)

Confounder:  [1 1 1 ... 0 1 1]
Variable to adjust for:  [0 0 1 ... 0 1 1]
Treatment:  [0 0 0 ... 0 0 1]
Output:  [2.47143516 1.47143516 2.47143516 ... 0.47143516 2.47143516 2.47143516]
Conditional probability diff:  -0.06480524642289343
Adjusted Formula:  -0.19764165902467057
theoretical ATE:  -0.2


| #   | Question                                        | Result |
|-----|-------------------------------------------------|--------|
| 1   | Estimated E(O\|T=1) - E(O\|T=0)                 | -0.06  |
| 2   | adjustment(1) - adjustment(0)                   | -0.198 |
| 3   | Theoretical ATE = E(O\|do(T=1)) - E(O\|do(T=0)) | -0.2   |
| 4   | Estimated ATE from data                         | -0.198 |

- An unobservable confounder, C, has an impact on the outcome and indirectly influences the treatment through Z.
- To mitigate the influence of C and satisfy the Backdoor Criterion, we must control for Z. This adjustment allows us to eliminate the confounding effect, ensuring a more accurate evaluation of the causal relationship between treatment (T) and outcome.

In [10]:
#EXERCISE B

seed (1234)
n = 10000

#U1 := B(0.5)
#U2 := B(0.3)
#T := B(0.4)U1
#Z := B(0.8) U1 + B(0.8) U2
#O := B(0.4) T + U2 + 𝜀
#B is a Bernoulli distribution, 𝜀 ~N(0,1)

# 𝜀 ~N(0,1)
epsilon = normal(0,1,size=n)

U1 = np.random.binomial(1,0.5,size=n)
U2 = np.random.binomial(1,0.3,size=n)
#T := B(0.6) Z + B(0.2)(1-Z)
T = np.random.binomial(1,0.4,size=n)*U1
#Z := B(0.8) U1 + B(0.8) U2
Z = np.random.binomial(1,0.8,size=n)*U1 + np.random.binomial(1,0.8,size=n)*U2
O = np.random.binomial(1,0.4,size=n)*(T) + U2 + epsilon


print('Confounder1: ', U1)
print('Confounder2: ', U2)
print('Treatment: ', T)
print('Variable to adjust for: ', Z)
print('Output: ', O)


#condition_prob_diff = mean(outcome[treatment==1]) - mean(outcome[treatment==0])
conditional_prob_diff = np.mean(O[T==1]) - np.mean(O[T==0])
print('Conditional probability diff: ',conditional_prob_diff)

#ADJUSTED FORMULA 

e_t1_z1 = np.mean(O[(T==1)&(Z==1)])

e_t1_z0 = np.mean(O[(T==1)&(Z==0)])

e_t0_z1 = np.mean(O[(T==0)&(Z==1)])

e_t0_z0 = np.mean(O[(T==0)&(Z==0)])

# Calculate the ate
adjusted_formula = (e_t1_z1 * np.mean(Z == 1) + e_t1_z0 * np.mean(Z == 0) ) - ( e_t0_z1 * np.mean(Z == 1) + e_t0_z0 * np.mean(Z == 0))
print('Adjusted Formula: ',adjusted_formula)


#Theoretical ATE 

eO_t1 = (0.4)*(1) 
eO_t0 = (0.4)*(0) 

Theoretical_ATE = eO_t1 - eO_t0

print('theoretical ATE: ',Theoretical_ATE)



Confounder1:  [1 1 1 ... 1 1 0]
Confounder2:  [0 0 0 ... 0 0 1]
Treatment:  [1 1 0 ... 1 0 0]
Variable to adjust for:  [1 1 0 ... 0 0 0]
Output:  [ 0.47143516 -0.19097569  1.43270697 ... -1.05246069 -0.4976931
  0.7439938 ]
Conditional probability diff:  0.357027520293951
Adjusted Formula:  0.20765546946542346
theoretical ATE:  0.4


| #   | Question                                        | Result |
|-----|-------------------------------------------------|--------|
| 1   | Estimated E(O\|T=1) - E(O\|T=0)                 | 0.357  |
| 2   | adjustment(1) - adjustment(0)                   | 0.207  |
| 3   | Theoretical ATE = E(O\|do(T=1)) - E(O\|do(T=0)) |  0.4   |
| 4   | Estimated ATE from data                         | 0.357  |

- In this scenario, Z does not influence the treatment, as there is no direct connection between them.

- Adjusting for Z in this case could potentially decrease the Average Treatment Effect (ATE), as it might introduce new connections between confounders and Treatment/Outcome.

- Therefore, the preferable approach would be to calculate the direct effect of treatment on the outcome, providing a more straightforward assessment of the causal relationship.

In [13]:
#EXERCISE C

seed (1234)
n = 10000

#U := B(0.4)
#Z := B(0.3)
#T := B(0.4) U Z+ B(0.7) (1 - Z)
#O := B(0.4) T + B(0.5) (1-T) + U + 𝜀

# 𝜀 ~N(0,1)
epsilon = normal(0,1,size=n)

U = np.random.binomial(1,0.4,size=n)
Z = np.random.binomial(1,0.3,size=n)
T = np.random.binomial(1,0.4,size=n)*U*Z + np.random.binomial(1,0.7,size=n)*(1-Z)
#O := B(0.4) T + B(0.5) (1-T) + U + 𝜀
O = np.random.binomial(1,0.4,size=n)*(T) + np.random.binomial(1,0.5,size=n)*(1-T) + U + epsilon

print('Confounder1: ', U)
print('Variable to adjust for: ', Z)
print('Treatment: ', T)
print('Output: ', O)

#condition_prob_diff = mean(outcome[treatment==1]) - mean(outcome[treatment==0])
conditional_prob_diff = np.mean(O[T==1]) - np.mean(O[T==0])
print('Conditional probability diff: ',conditional_prob_diff)

#ADJUSTED FORMULA 

e_t1_z1 = np.mean(O[(T==1)&(Z==1)])

e_t1_z0 = np.mean(O[(T==1)&(Z==0)])

e_t0_z1 = np.mean(O[(T==0)&(Z==1)])

e_t0_z0 = np.mean(O[(T==0)&(Z==0)])

# Calculate the ate
adjusted_formula = (e_t1_z1 * np.mean(Z == 1) + e_t1_z0 * np.mean(Z == 0) ) - ( e_t0_z1 * np.mean(Z == 1) + e_t0_z0 * np.mean(Z == 0))
print('Adjusted Formula: ',adjusted_formula)


#Theoretical ATE = (E[O|T==1])

eO_t1 = (0.4)*(1) + (0.5)*(0)
eO_t0 = (0.4)*(0) + (0.5)*(1)

Theoretical_ATE = eO_t1 - eO_t0

print('theoretical ATE: ',Theoretical_ATE)

Confounder1:  [1 1 0 ... 0 1 0]
Variable to adjust for:  [0 0 0 ... 0 0 1]
Treatment:  [1 1 0 ... 0 0 0]
Output:  [ 2.47143516  0.80902431  1.43270697 ... -1.05246069  1.5023069
  0.7439938 ]
Conditional probability diff:  0.04040644263397719
Adjusted Formula:  0.13419922370240434
theoretical ATE:  -0.09999999999999998


| #   | Question                                        | Result |
|-----|-------------------------------------------------|--------|
| 1   | Estimated E(O\|T=1) - E(O\|T=0)                 |  0.04|
| 2   | adjustment(1) - adjustment(0)                   |  0.134 |
| 3   | Theoretical ATE = E(O\|do(T=1)) - E(O\|do(T=0)) | -0.1   |
| 4   | Estimated ATE from data                         |    IV  |

- In this scenario, the application of an instrumental variable becomes necessary to effectively control for the confounding effect.

- Adjusting for Z or directly calculating the Average Treatment Effect (ATE) may not yield an accurate estimate of the true ATE in this situation.

In [12]:
#EXERCISE D

seed (1234)
n = 10000


#T := B(0.8)
#M := B(0.6) T + B(0.2)(1-T)
#O := B(0.3) M + B(0.5) (1-M) + 𝜀
#Z := B(0.1) M + B(0.9)(1-M)

# 𝜀 ~N(0,1)
epsilon = normal(0,1, size=n)

T = np.random.binomial(1,0.8,size=n)
M = np.random.binomial(1,0.6,size=n)*T + np.random.binomial(1,0.2,size=n)*(1-T)
#O := B(0.4) T + B(0.5) (1-T) + U + 𝜀
O = np.random.binomial(1,0.3,size=n)*M + np.random.binomial(1,0.5,size=n)*(1-M) + epsilon
O_c = np.random.binomial(1,0.3,size=n)*(np.random.binomial(1,0.6,size=n)*T + np.random.binomial(1,0.2,size=n)*(1-T)) + np.random.binomial(1,0.5,size=n)*(1-(np.random.binomial(1,0.6,size=n)*T + np.random.binomial(1,0.2,size=n)*(1-T))) + epsilon
Z = np.random.binomial(1,0.1,size=n)*M + np.random.binomial(1,0.9,size=n)*(1-M)

print('Mediator: ', M)
print('Variable to adjust for: ', Z)
print('Treatment: ', T)
print('Output: ', O)

#condition_prob_diff = mean(outcome[treatment==1]) - mean(outcome[treatment==0])

conditional_prob_diff = np.mean(O_c[T==1]) - np.mean(O_c[T==0])

print('Conditional probability diff: ',conditional_prob_diff)

#ADJUSTED FORMULA 

e_t1_z1 = np.mean(O_c[(T==1)&(Z==1)])
e_t1_z0 = np.mean(O_c[(T==1)&(Z==0)])
e_t0_z1 = np.mean(O_c[(T==0)&(Z==1)])
e_t0_z0 = np.mean(O_c[(T==0)&(Z==0)])

# Calculate the ate
adjusted_formula = (e_t1_z1 * np.mean(Z == 1) + e_t1_z0 * np.mean(Z == 0) ) - ( e_t0_z1 * np.mean(Z == 1) + e_t0_z0 * np.mean(Z == 0))
print('Adjusted Formula: ',adjusted_formula)


#Theoretical ATE
eO_t1 = (0.3) * ((0.6)* 1 + (0.2)*(1-1)) + (0.5) * (1-((0.6)* 1 + (0.2)*(1-1)))
eO_t0 = (0.3) * ((0.6)* 0 + (0.2)*(1-0)) + (0.5) * (1-((0.6)* 0 + (0.2)*(1-0)))


Theoretical_ATE = eO_t1 - eO_t0
print('Theoretical_ATE',Theoretical_ATE)

Mediator:  [1 1 1 ... 1 1 0]
Variable to adjust for:  [0 0 0 ... 0 0 1]
Treatment:  [1 0 1 ... 1 1 1]
Output:  [ 0.47143516 -1.19097569  2.43270697 ... -0.05246069  0.5023069
  0.7439938 ]
Conditional probability diff:  -0.09944685634781614
Adjusted Formula:  -0.10647752538812771
Theoretical_ATE -0.08000000000000002


In [7]:
print(np.mean(O[T==1]))
print(np.mean(O[(T==1)&(Z==1)]))

0.39616133290118866
0.48914714151979005


| #   | Question                                        | Result |
|-----|-------------------------------------------------|--------|
| 1   | Estimated E(O\|T=1) - E(O\|T=0)                 |  -0.0994 |
| 2   | adjustment(1) - adjustment(0)                   |  -0.1064 |
| 3   | Theoretical ATE = E(O\|do(T=1)) - E(O\|do(T=0)) |  -0.08 |
| 4   | Estimated ATE from data                         |   -0.0994 |

- There is no need to adjust for Z since it does not have any influence on either the treatment (T) or the outcome (O).
- Considering the causal direction from M to Z, controlling for Z would not affect M. Therefore, controlling for Z should not block the path between T and O.
- To calculate the Average Treatment Effect (ATE) of T on O, we must do so through the mediator (M). This involves multiplying the effect of T on M by the effect of M on O.

### PART 2



Intervening is not conditioning. Calculate (by hand) the distributions:
C --> E

C := εC  
E := C +  εE  
εC, εE ~ N(0, 1)  


### E|do(C:=2) and E|C=2

Calculation:

#### E|do(C:=2) means that we need to intervine and set C = 2  

E(E|doC=2) = E(2 + εE)    
    =   2 + E(εE) because of strict exogeneity,the expectation of εE~(0,1) is 0    
    =   2 + 0    
    =   2  Mean
         
Var(E|doC=2) = Var(2 + εE)    
           = 0 + Var(εE) since εE~N(0,1), the variance of εE is equal to 1    
           = 1 Variance    
  
  
#### E|C=2 conditional expectation of E when C=2    

E(E|C=2) = E(2 + εE)    
         =   2 + E(εE) because of strict exogeneity, the expectation of εE~(0,1) is 0    
         =   2 + 0    
         =   2  Mean    
         
Var(E|C=2) = Var(2 + εE)    
           = 0 + Var(εE) since εE~N(0,1), the variance of εE is equal to 1    
           = 1 Variance    

---


C := εC  
E := C + εE   
εC, εE ~ N(0, 1)   

### C|do(E:=2) and C|E=2

Calculation:

#### C|do(E:=2) means that we need to intervine and set E = 2

E(C|doE=2) = E(εC) since εC~N(0,1), the expectation would be 0  
           = 0 mean    
           
Var(C|doE=2) = Var(εC) since εC~N(0,1), the variance of εE is equal to 1    
             = 1 Variance    


#### C|E=2 conditional expectation of C when E=2   

E(C|E=2)    
E=2  -->  C= 2 + εC 
       
E(C|E=2) = E(2) - E(εC)    
         = 2 - 0    
         = 2    

Var(C|E=2) = Var(2) - Var(εC)       
           = 1 - 0    
           = 1    


Conclusions:


- Intervening on E and conditioning on E can lead to different outcomes because intervening actively changes the variable and the system structure, while conditioning only observes the system under a specific condition.

- In this particular scenario, when intervening we are cutting the incoming rows and sets E to 2.
- Conditioning on the other hand respects the causal structure (causal relation) setting E to 2. It is expected to receive different outcomes

           
